# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
spark

#### Passing other options to spark session:
    
    

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config('someconfigkey', 'someconfigvalue').getOrCreate()

spark.sparkContext.getConf().getAll()

[('spark.driver.port', '38997'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('someconfigkey', 'someconfigvalue'),
 ('spark.app.id', 'local-1556963107310')]

We can check option values in the resulting session like this:

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [26]:
import random

random.seed(42)

data = [ (id_, random.choice(['drarf', 'elf', 'orc', 'human'])) for id_ in range(20)]

df = spark.createDataFrame(data, schema=['id', 'species'])

In [21]:
df.show(4)

+---+-------+
| id|species|
+---+-------+
|  0|  drarf|
|  1|  drarf|
|  2|    orc|
|  3|    elf|
+---+-------+
only showing top 4 rows



In [16]:
from pyspark.sql import Row

help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  Row(*args, **kwargs)
 |  
 |  A row in L{DataFrame}.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments,
 |  the fields will be sorted by names. It is not allowed to omit
 |  a named argument to represent the value is None or missing. This should be
 |  explicitly set to None in this case.
 |  
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(age=11, name='Alice')
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  False
 |  
 |  Row also can be used to create another Row like class, then it
 |  could be used to create Row objects, such as
 |  
 |  >>> Person = Row("name", "age")
 |  >>> Person
 |  <R

In [22]:
first_row =  df.first()
first_row.id

0

In [24]:
first_row['id']

0

In [27]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- species: string (nullable = true)



### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [28]:
coupon = sc.textFile('coupon150720.csv')

def getValues(line):
    fields = line.split(',')
    return (fields[0], fields[2], fields[3], fields[4], float(fields[6]))

coupon_f = coupon.map(getValues)

In [30]:
coupons_df = spark.createDataFrame(coupon_f)
coupons_df.show(5)

+--------------+---+---+---+------+
|            _1| _2| _3| _4|    _5|
+--------------+---+---+---+------+
|79062005698500|MAA|AUH| 9W| 56.79|
|79062005698500|AUH|CDG| 9W| 84.34|
|79062005924069|CJB|MAA| 9W|  60.0|
|79065668570385|DEL|DXB| 9W|160.63|
|79065668737021|AUH|IXE| 9W|152.46|
+--------------+---+---+---+------+
only showing top 5 rows



### Inferring and specifying schemas

In [31]:
coupons_df = spark.createDataFrame(coupon_f, schema=['tkt_nuber', 'origin', 'destination', 'airline', 'amount'])
coupons_df.show(5)

+--------------+------+-----------+-------+------+
|     tkt_nuber|origin|destination|airline|amount|
+--------------+------+-----------+-------+------+
|79062005698500|   MAA|        AUH|     9W| 56.79|
|79062005698500|   AUH|        CDG|     9W| 84.34|
|79062005924069|   CJB|        MAA|     9W|  60.0|
|79065668570385|   DEL|        DXB|     9W|160.63|
|79065668737021|   AUH|        IXE|     9W|152.46|
+--------------+------+-----------+-------+------+
only showing top 5 rows



In [32]:
coupons_df.printSchema()

root
 |-- tkt_nuber: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- amount: double (nullable = true)



#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [42]:
from pyspark.sql import types

shema = types.StructType([types.StructField('tkt_number', types.StringType(), nullable=False),
                 types.StructField('origin', types.StringType(), nullable=True),
                  types.StructField('destination', types.StringType(), nullable=True),
                  types.StructField('airline', types.StringType(), nullable=True),
                  types.StructField('amount', types.DoubleType(), nullable=True),
                 
                 ])

shema

StructType(List(StructField(tkt_number,StringType,false),StructField(origin,StringType,true),StructField(destination,StringType,true),StructField(airline,StringType,true),StructField(amount,DoubleType,true)))

In [43]:
types.StructType()

StructType(List())

In [44]:
coupons_df = spark.createDataFrame(coupon_f, shema)
coupons_df.show(5)

+--------------+------+-----------+-------+------+
|    tkt_number|origin|destination|airline|amount|
+--------------+------+-----------+-------+------+
|79062005698500|   MAA|        AUH|     9W| 56.79|
|79062005698500|   AUH|        CDG|     9W| 84.34|
|79062005924069|   CJB|        MAA|     9W|  60.0|
|79065668570385|   DEL|        DXB|     9W|160.63|
|79065668737021|   AUH|        IXE|     9W|152.46|
+--------------+------+-----------+-------+------+
only showing top 5 rows



In [45]:
coupons_df.printSchema()

root
 |-- tkt_number: string (nullable = false)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- amount: double (nullable = true)



#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [49]:
df2 = spark.read.csv('coupon150720.csv', inferSchema=True)
df2.show(5)
df2.printSchema()

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows

root
 |-- _c0: long (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: doub

In [54]:
flight_mad = spark.sql('''SELECT _c0 as tkt_number,
            _c2 as origin,
            _c3 as destination,
            _c4 as airlines,
            CAST(_c6 as double) as amount
          FROM csv.`coupon150720.csv`
          where _c3=\'MAD\'    ''')

flight_mad.show(5)

+--------------+------+-----------+--------+------+
|    tkt_number|origin|destination|airlines|amount|
+--------------+------+-----------+--------+------+
|79062005639642|   BRU|        MAD|      UX| 21.02|
|79065668754871|   BRU|        MAD|      SN| 27.66|
|79065668917696|   CDG|        MAD|      AF| 46.97|
|79062006133090|   CDG|        MAD|      AF|  3.38|
|79062006110497|   CDG|        MAD|      AF| 26.02|
+--------------+------+-----------+--------+------+
only showing top 5 rows



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [ ]:
spark.read.json
sparj.read.parquet

### Basic operations with DataFrames

In [55]:
df.show(5)

+---+-------+
| id|species|
+---+-------+
|  0|  drarf|
|  1|  drarf|
|  2|    orc|
|  3|    elf|
|  4|    elf|
+---+-------+
only showing top 5 rows



### Filtering and selecting

Syntax inspired in SQL.

In [59]:
df.select('species').show(5)

+-------+
|species|
+-------+
|  drarf|
|  drarf|
|    orc|
|    elf|
|    elf|
+-------+
only showing top 5 rows



+---+-------+
| id|species|
+---+-------+
|  3|    elf|
|  4|    elf|
|  5|    elf|
| 12|    elf|
| 13|    elf|
+---+-------+
only showing top 5 rows



+---+-------+
| id|species|
+---+-------+
|  3|    elf|
|  4|    elf|
|  5|    elf|
| 12|    elf|
| 13|    elf|
+---+-------+
only showing top 5 rows



If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [66]:
df.filter(df['species'] == 'elf').show(5)

+---+-------+
| id|species|
+---+-------+
|  3|    elf|
|  4|    elf|
|  5|    elf|
| 12|    elf|
| 13|    elf|
+---+-------+
only showing top 5 rows



That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

+---+-------+
| id|species|
+---+-------+
|  3|    elf|
|  4|    elf|
|  5|    elf|
| 12|    elf|
| 13|    elf|
+---+-------+
only showing top 5 rows



`where` is exactly synonimous with `filter`

In [ ]:
df.where(df['species'] == 'elf').show(5)

A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

#### Exercise

Extract all employee ids which correspond to managers

In [76]:
df.where(df['species'] == 'orc').select('id').show(5)

+---+
| id|
+---+
|  2|
| 19|
+---+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [77]:
df['weapon'] = 'axe'

TypeError: 'DataFrame' object does not support item assignment

In [79]:
df2 = df.withColumn('id_squared', df['id'] ** 2)
df2.show(6)

+---+-------+----------+
| id|species|id_squared|
+---+-------+----------+
|  0|  drarf|       0.0|
|  1|  drarf|       1.0|
|  2|    orc|       4.0|
|  3|    elf|       9.0|
|  4|    elf|      16.0|
|  5|    elf|      25.0|
+---+-------+----------+
only showing top 6 rows



In [83]:
df2 = df.select('id', 'species', (df['id'] ** 2).cast(types.IntegerType()).alias('id_squared'))
df2.show(6)

+---+-------+----------+
| id|species|id_squared|
+---+-------+----------+
|  0|  drarf|         0|
|  1|  drarf|         1|
|  2|    orc|         4|
|  3|    elf|         9|
|  4|    elf|        16|
|  5|    elf|        25|
+---+-------+----------+
only showing top 6 rows



### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [86]:
from pyspark.sql import functions

df2 = df.select('id', 'species', (functions.log(df['id'])).alias('id_squared'))
df2.show(6)

+---+-------+------------------+
| id|species|        id_squared|
+---+-------+------------------+
|  0|  drarf|              null|
|  1|  drarf|               0.0|
|  2|    orc|0.6931471805599453|
|  3|    elf|1.0986122886681098|
|  4|    elf|1.3862943611198906|
|  5|    elf|1.6094379124341003|
+---+-------+------------------+
only showing top 6 rows



This errors out because 

```python
math.log1p
```

is not a udf: it doesn't know how to work with strings or Column objects:

In [88]:
import math

df2 = df.select('id', 'species', (math.log(df['id'])).alias('id_squared'))
df2.show(6)

TypeError: must be real number, not Column

But we can transform it into a udf:

In [91]:
panolizador =  functions.udf(lambda word: word + ' panoli')

df2 = df.select('id', 'species', (panolizador(df['species'])).alias('id_squared'))
df2.show(6)

+---+-------+------------+
| id|species|  id_squared|
+---+-------+------------+
|  0|  drarf|drarf panoli|
|  1|  drarf|drarf panoli|
|  2|    orc|  orc panoli|
|  3|    elf|  elf panoli|
|  4|    elf|  elf panoli|
|  5|    elf|  elf panoli|
+---+-------+------------+
only showing top 6 rows



We can do the same with any function we dream up:

In [96]:
def firstWord(word):
    return word[0]

firt_letter = functions.udf(firstWord)

df2 = df.select('id', 'species', (firt_letter(df['species'])))
df2.show(6)

+---+-------+------------------+
| id|species|firstWord(species)|
+---+-------+------------------+
|  0|  drarf|                 d|
|  1|  drarf|                 d|
|  2|    orc|                 o|
|  3|    elf|                 e|
|  4|    elf|                 e|
|  5|    elf|                 e|
+---+-------+------------------+
only showing top 6 rows



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [101]:
def loglp(n): return math.log(1 +n)

loglp_udf = functions.udf(loglp, types.DoubleType())

df2 = df.select('id', 'species', (loglp_udf(df['id'])))
df2.show(6)
df2.printSchema()

+---+-------+------------------+
| id|species|         loglp(id)|
+---+-------+------------------+
|  0|  drarf|               0.0|
|  1|  drarf|0.6931471805599453|
|  2|    orc|1.0986122886681098|
|  3|    elf|1.3862943611198906|
|  4|    elf|1.6094379124341003|
|  5|    elf| 1.791759469228055|
+---+-------+------------------+
only showing top 6 rows

root
 |-- id: long (nullable = true)
 |-- species: string (nullable = true)
 |-- loglp(id): double (nullable = true)



Think about this function: what is its return type?

#### Exercise: 

Create a 'salary' field in our df. make it 30000 for mechanics, 40000 for salespeople and 70000 for managers.



In [117]:
def weight(specie):
    if specie == 'drarf':
        return 35
    if specie == 'elf':
        return 50
    if specie == 'human':
        return 70
    if specie == 'orc':
        return 120

weight_udf = functions.udf(weight)

df2 = df.select('id', 'species', (weight_udf(df['species'])).alias('weight').cast(types.IntegerType()))
df2.show(5)

+---+-------+------+
| id|species|weight|
+---+-------+------+
|  0|  drarf|    35|
|  1|  drarf|    35|
|  2|    orc|   120|
|  3|    elf|    50|
|  4|    elf|    50|
+---+-------+------+
only showing top 5 rows



If we have a column that is not the desired type, we can convert it with `cast`.

### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [120]:
df2.summary().show()

+-------+-----------------+-------+----------------+
|summary|               id|species|          weight|
+-------+-----------------+-------+----------------+
|  count|               20|     20|              20|
|   mean|              9.5|   null|            54.0|
| stddev|5.916079783099616|   null|25.6289310206875|
|    min|                0|  drarf|              35|
|    25%|                4|   null|              35|
|    50%|                9|   null|              50|
|    75%|               14|   null|              50|
|    max|               19|    orc|             120|
+-------+-----------------+-------+----------------+



In [121]:
df2.describe().show()

+-------+-----------------+-------+----------------+
|summary|               id|species|          weight|
+-------+-----------------+-------+----------------+
|  count|               20|     20|              20|
|   mean|              9.5|   null|            54.0|
| stddev|5.916079783099616|   null|25.6289310206875|
|    min|                0|  drarf|              35|
|    max|               19|    orc|             120|
+-------+-----------------+-------+----------------+



### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [125]:
def aligment(): return random.choice(['good','evil'])
aligment_udf = functions.udf(aligment)

df3 = df2.withColumn('alignment', aligment_udf())
df3.show(5)

+---+-------+------+---------+
| id|species|weight|alignment|
+---+-------+------+---------+
|  0|  drarf|    35|     evil|
|  1|  drarf|    35|     evil|
|  2|    orc|   120|     good|
|  3|    elf|    50|     good|
|  4|    elf|    50|     evil|
+---+-------+------+---------+
only showing top 5 rows



In [126]:
df3.crosstab('species', 'alignment').show()

+-----------------+----+----+
|species_alignment|evil|good|
+-----------------+----+----+
|              orc|   2|   0|
|            drarf|   2|   6|
|            human|   0|   3|
|              elf|   5|   2|
+-----------------+----+----+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [135]:
groups = df3.groupBy('alignment')

GroupedData has several aggregation functions defined:

In [136]:
groups.avg('weight').show()

+---------+------------------+
|alignment|       avg(weight)|
+---------+------------------+
|     evil| 55.90909090909091|
|     good|51.666666666666664|
+---------+------------------+



We can do several aggregations in a single step, with a number of different syntaxes:

In [137]:
groups.min('weight').show()

+---------+-----------+
|alignment|min(weight)|
+---------+-----------+
|     evil|         35|
|     good|         35|
+---------+-----------+



In [140]:
groups.agg(functions.min('weight').alias('min_wt'), functions.stddev('weight').alias('std')).show()

+---------+------+------------------+
|alignment|min_wt|               std|
+---------+------+------------------+
|     evil|    35| 33.43318378165289|
|     good|    35|11.254628677422753|
+---------+------+------------------+



In [141]:
groups.agg({'weight' :'mean', 'id': 'max'}).show()

+---------+-----------------+-------+
|alignment|      avg(weight)|max(id)|
+---------+-----------------+-------+
|     evil|53.57142857142857|     18|
|     good|54.23076923076923|     19|
+---------+-----------------+-------+



### Intersections

Ver much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
session.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

We can erase it with `unpersist`

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

1) Calculate the mean and std of salary for each location

2) Annotate each employee with the stats corresponding to their location

3) Calculate the z-score

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

Once registered, we can perform queries as complex as we want.

#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

## Writing out


#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

2) Total ticket amounts per origin

3) Top 10 Airlines by average amount



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf